In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from random import choice, sample
from joblib import Parallel, delayed
import GPminer as GPm
import FreeBack as FB
import time, datetime, os, re, shutil

# 数据

In [6]:
start = '2024-1-1'
end = '2024-9-1'

market = pd.read_csv('./market.csv.xz', parse_dates=['date']).set_index(['date', 'code']).loc[start:end] # 市场数据
benchmark = pd.read_csv('./benchmark.csv.xz', parse_dates=['date']).set_index(['date']).loc[start:end] # 指数数据

# 评级转化为连续因子
market['rating'] = market['rating'].fillna(market['rating'].mode().iloc[0]) # 众数填充
replace_dict = {'AAA': 0, 'AA+': 1, 'AA': 2, 'AA-': 3, 'A+': 4, 'A': 5, 'A-': 6, 'BBB+': 7, 'BBB': 8, 'BBB-': 9, 'BB+': 10, 'BB': 11, 'BB-': 12, 'B+': 13, 'B': 14, 'B-': 15, 'CCC': 16, 'CC': 17, 'C':18}
market['rating'] = market['rating'].replace(replace_dict)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4720\4074347238.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  market['rating'] = market['rating'].replace(replace_dict)


# 遗传算法参数

In [4]:
fitness = 'sharpe' # 适应度目标
population_size = 10 # 种群规模
evolution_ratio = 0.2 # 每一代选择比例
prob_dict = {} # 全部变异算子等概率运行
select_alg = 'cut' # 子代选择方法 截断 cut  锦标赛 tournament
n_core = 4  # 并行计算核数，1代表串行
max_g = 30 # 最大进化代数，操作此代数结束进化
tolerance_g = 3 # 最大允许适应度不提升代数，适应多连续不提升超过此代数结束进化
n_batch = 3  # 进行n_batch组初始种群的进化

# Score打分因子挖掘工作流，以平衡性转债为例

In [3]:
##### 策略超参数 #####
# 所有策略共用一个选股池平衡性转债
pool_share = GPm.ind.Pool(";list_days>1800|close<100|close>135|is_call=公告实施强赎,公告提示强赎,公告到期赎回")
#GPm.ind.Score.max_exp_len = 10  # 生成的策略最大因子数量（默认10因子）
#GPm.ind.Score.max_mul = 50 # 因子最大系数（默认50）
# 因子库
score_basket = ['close', 'conv_prem', 'bond_prem', 'mod_conv_prem', 'dblow',\
                 'list_days', 'volatility', 'volatility_stk', 'remain_size', 'pe_ttm', 'pb']
# 给出一个初始种子在此基础上变异生成初始种子,如果没有初始score的话则随机生成种子
#p0 = GPm.ind.Score("1*True*volatility+1*False*dblow")
p0 = None
hold_num = 5  # 持仓数量
price = 'close' # 结算价格
code_returns = None # 标的收益（默认None标的收益等于close到close的变化率）
comm = 10/1e4 # 单边交易成本

# 运行

In [5]:
miner0 = GPm.work.Miner(market, benchmark, pool_share, score_basket=score_basket, p0=p0,\
                hold_num=hold_num, price=price, code_returns=code_returns, comm=comm)
miner0.prepare(fitness=fitness, population_size=population_size, evolution_ratio=evolution_ratio,\
                prob_dict=prob_dict, select_alg=select_alg, n_core=n_core,\
                max_g=max_g, tolerance_g=tolerance_g)
for i in range(n_batch):
    miner0.run()

# Pool排除因子挖掘工作流，以转股溢价率为例